In [3]:
# --- 环境设置 ---
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

print("=== MACD指标完整计算示例 ===\n")

# --- 数据准备 ---
# 创建30天的模拟股票数据
df = pd.DataFrame({
    'time': pd.date_range('2024-01-01', periods=30),
    'price': [100.00, 101.50, 102.20, 101.80, 103.10, 104.50, 103.90, 105.20, 106.00, 105.50,
              107.20, 108.10, 107.80, 109.30, 108.90, 110.20, 111.50, 110.80, 112.30, 113.00,
              112.50, 114.20, 115.10, 114.60, 116.30, 115.80, 117.50, 118.20, 117.90, 119.40]
})

print("原始数据:")
print(df.head(10))
print("...")
print(df.tail(10))
print()


# --- 步骤1：计算EMA函数 ---
def calculate_ema(prices, period):
    """
    计算指数移动平均线
    公式: EMA(今日) = α × 价格(今日) + (1-α) × EMA(昨日)
    其中: α = 2 / (period + 1)
    """
    alpha = 2 / (period + 1)
    print(f"计算{period}日EMA，平滑因子α = 2/({period}+1) = {alpha:.4f}")

    ema = [prices.iloc[0]]  # 第一天用价格本身

    for i in range(1, len(prices)):
        if i < period:
            # 前period天用简单移动平均作为初始值
            ema_value = prices.iloc[:i + 1].mean()
        else:
            # 使用EMA公式
            ema_value = alpha * prices.iloc[i] + (1 - alpha) * ema[i - 1]

        ema.append(ema_value)

    return pd.Series(ema, index=prices.index)


# --- 步骤2：计算12日EMA ---
print("步骤1: 计算12日EMA")
df['ema12'] = calculate_ema(df['price'], 12)
print("12日EMA计算完成\n")

# --- 步骤3：计算26日EMA ---
print("步骤2: 计算26日EMA")
df['ema26'] = calculate_ema(df['price'], 26)
print("26日EMA计算完成\n")

# --- 步骤4：计算MACD线 ---
print("步骤3: 计算MACD线")
df['macd'] = df['ema12'] - df['ema26']
print("MACD = EMA12 - EMA26")
print("MACD计算完成\n")

# --- 步骤5：计算信号线（9日EMA of MACD）---
print("步骤4: 计算信号线（MACD的9日EMA）")
# 只对有MACD值的部分计算信号线
macd_values = df['macd'].dropna()
if len(macd_values) > 0:
    signal_values = calculate_ema(macd_values, 9)
    df['signal'] = np.nan
    df.loc[macd_values.index, 'signal'] = signal_values
print("信号线计算完成\n")

# --- 步骤6：计算柱状图 ---
print("步骤5: 计算柱状图")
df['histogram'] = df['macd'] - df['signal']
print("Histogram = MACD - Signal")
print("柱状图计算完成\n")

# --- 结果展示 ---
print("=== 完整结果表 ===")
print("前15天（EMA构建期）:")
result_cols = ['time', 'price', 'ema12', 'ema26', 'macd', 'signal', 'histogram']
print(df[result_cols].head(15).round(2))

print("\n后15天（包含完整MACD信号）:")
print(df[result_cols].tail(15).round(2))

# --- 只显示有完整MACD信号的日期 ---
print("\n=== 有效MACD信号期（第26天开始）===")
valid_macd = df[df['signal'].notna()].copy()
if not valid_macd.empty:
    print(f"有效MACD数据点: {len(valid_macd)} 个")
    print(valid_macd[result_cols].round(2))
else:
    print("数据不足，无法生成完整的MACD信号")

# --- 手工验证关键计算 ---
print("\n=== 手工验证计算 ===")
if len(df) >= 26:
    day26_idx = 25  # 第26天，索引为25
    print(f"第26天验证:")
    print(f"价格: {df.iloc[day26_idx]['price']}")
    print(f"EMA12: {df.iloc[day26_idx]['ema12']:.2f}")
    print(f"EMA26: {df.iloc[day26_idx]['ema26']:.2f}")
    print(f"MACD: {df.iloc[day26_idx]['macd']:.2f}")

    if not pd.isna(df.iloc[day26_idx]['signal']):
        print(f"Signal: {df.iloc[day26_idx]['signal']:.2f}")
        print(f"Histogram: {df.iloc[day26_idx]['histogram']:.2f}")

# --- 交易信号分析 ---
print("\n=== 交易信号分析 ===")
valid_data = df.dropna(subset=['histogram'])

if not valid_data.empty:
    # 寻找金叉死叉信号
    for i in range(1, len(valid_data)):
        current_hist = valid_data.iloc[i]['histogram']
        previous_hist = valid_data.iloc[i - 1]['histogram']
        date = valid_data.iloc[i]['time'].strftime('%Y-%m-%d')
        price = valid_data.iloc[i]['price']

        if previous_hist <= 0 and current_hist > 0:
            print(f"🔥 金叉信号: {date}, 价格: {price}, 柱状图: {current_hist:.2f}")
        elif previous_hist >= 0 and current_hist < 0:
            print(f"💀 死叉信号: {date}, 价格: {price}, 柱状图: {current_hist:.2f}")

    # 总体趋势判断
    final_macd = valid_data.iloc[-1]['macd']
    final_hist = valid_data.iloc[-1]['histogram']

    print(f"\n当前趋势判断:")
    print(f"MACD值: {final_macd:.2f} ({'看多' if final_macd > 0 else '看空'})")
    print(f"柱状图: {final_hist:.2f} ({'上升动能' if final_hist > 0 else '下降动能'})")

# --- 保存为CSV ---
print(f"\n=== 数据保存 ===")
df.round(2).to_csv('macd_calculation_result.csv', index=False)
print("结果已保存为: macd_calculation_result.csv")


# --- 快速测试函数 ---
def quick_macd_analysis(price, period_fast=12, period_slow=26, period_signal=9):
    """快速MACD分析函数"""
    if len(price) < period_slow:
        return "数据不足，需要至少{}天数据".format(period_slow)

    ema_fast = price.ewm(span=period_fast).mean()
    ema_slow = price.ewm(span=period_slow).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=period_signal).mean()
    histogram = macd_line - signal_line

    return pd.DataFrame({
        'macd': macd_line,
        'signal': signal_line,
        'histogram': histogram
    }).round(2)


print(f"\n=== 使用pandas内置函数验证 ===")
quick_result = quick_macd_analysis(df['price'])
quick_result

=== MACD指标完整计算示例 ===

原始数据:
        time  price
0 2024-01-01  100.0
1 2024-01-02  101.5
2 2024-01-03  102.2
3 2024-01-04  101.8
4 2024-01-05  103.1
5 2024-01-06  104.5
6 2024-01-07  103.9
7 2024-01-08  105.2
8 2024-01-09  106.0
9 2024-01-10  105.5
...
         time  price
20 2024-01-21  112.5
21 2024-01-22  114.2
22 2024-01-23  115.1
23 2024-01-24  114.6
24 2024-01-25  116.3
25 2024-01-26  115.8
26 2024-01-27  117.5
27 2024-01-28  118.2
28 2024-01-29  117.9
29 2024-01-30  119.4

步骤1: 计算12日EMA
计算12日EMA，平滑因子α = 2/(12+1) = 0.1538
12日EMA计算完成

步骤2: 计算26日EMA
计算26日EMA，平滑因子α = 2/(26+1) = 0.0741
26日EMA计算完成

步骤3: 计算MACD线
MACD = EMA12 - EMA26
MACD计算完成

步骤4: 计算信号线（MACD的9日EMA）
计算9日EMA，平滑因子α = 2/(9+1) = 0.2000
信号线计算完成

步骤5: 计算柱状图
Histogram = MACD - Signal
柱状图计算完成

=== 完整结果表 ===
前15天（EMA构建期）:
         time  price   ema12   ema26  macd  signal  histogram
0  2024-01-01  100.0  100.00  100.00  0.00    0.00       0.00
1  2024-01-02  101.5  100.75  100.75  0.00    0.00       0.00
2  2024-01-03  102.2  101

,macd,signal,histogram
0,0.00,0.00,0.00
1,0.03,0.02,0.01
2,0.06,0.04,0.03
3,0.06,0.05,0.02
4,0.11,0.07,0.05
5,0.20,0.10,0.10
6,0.22,0.13,0.09
7,0.30,0.17,0.13
8,0.39,0.22,0.16
9,0.42,0.27,0.15
